In [ ]:
# Projet issu des données suivantes :

# https://www.kaggle.com/datasets/iabhishekofficial/mobile-price-classification

In [2]:
# Importation des library

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
# Import des données

df_train = pd.read_csv('train.csv', sep = ',')
df_train.head()

FileNotFoundError: [Errno 2] No such file or directory: 'train.csv'

In [7]:
# Obtenir les noms des variables
noms_variables = df_train.columns

# Afficher les noms des variables
print(noms_variables)

Index(['battery_power', 'blue', 'clock_speed', 'dual_sim', 'fc', 'four_g',
       'int_memory', 'm_dep', 'mobile_wt', 'n_cores', 'pc', 'px_height',
       'px_width', 'ram', 'sc_h', 'sc_w', 'talk_time', 'three_g',
       'touch_screen', 'wifi', 'price_range'],
      dtype='object')


On décide de modifier le nom de nos variables ainsi que des modalités de la variable à expliquer

In [ ]:
# modification des noms des variables

df_train.rename(columns={'battery_power': 'puissance_batterie', 
                         'blue': 'Bluetooth', 
                         'clock_speed': 'vitesse_micro_pross', 
                         'fc': 'nb_pixels_camera_front', 
                         'four_g': '4G', 
                         'int_memory': 'mémoire_interne',
                         'm_dep': 'profondeur_tel',
                         'mobile_wt': 'poids_tel',
                         'n_cores': 'nb_coeurs',
                         'pc': 'nb_pixels_camera_externe',
                         'px_height': 'pixels_resolution_hauteur',
                         'px_width': 'pixels_resolution_largeur',
                         'sc_h': 'hauteur_tel',
                         'sc_w': 'largeur_tel',
                         'talk_time': 'duree_batterie_appel',
                         'three_g': '3G',
                         'touch_screen': 'ecran_tactile'})

In [ ]:
# Remplacer les modalités dans la colonne
nouveaux_noms_modalites = {0: 'bas de gamme', 1: 'prix moyen', 2: 'prix élevé', 3: 'haut de gamme'}
df_train['price_range'] = df_train['price_range'].replace(nouveaux_noms_modalites)

# Afficher les modalités de la colonne
modalites = df_train['price_range'].unique()

# Afficher les modalités
print("Modalités de la price_range :", modalites)

In [ ]:
# On sépare la variable à expliquer des variables explicatives
X = df_train.drop("price_range",axis=1)
y = df_train['price_range']

In [ ]:
# Nous allons utiliser cette livrary pour split nos échantillons
from sklearn.model_selection import train_test_split

In [ ]:
# On split le train et le test à 80 % et 20 %
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=0)

In [ ]:
# Import de la library pour utiliser le scaler
from sklearn.preprocessing import StandardScaler

In [ ]:
# On prévoit d'utiliser le scaler pour centrer et réduire les variables quantitatives
scaler = StandardScaler()

In [ ]:
# On applique le scaler à nos échantillons
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
# Importation de la library pour la regression logistique
from sklearn.linear_model import LogisticRegression

In [ ]:
# Application de la regression logistique sur notre échantillon d'apprentissage
model = LogisticRegression()
model.fit(X_train,y_train)

In [1]:
# Importation des library pour la pipeline
from sklearn.pipeline import make_pipeline

# Création de la pipeline
pLg = make_pipeline(StandardScaler(),
                    LogisticRegression(max_iter=1000,C=0.1)) #c = force de régularisation, a mettre dans un gridcv
pLg.fit(X_train, y_train)
print(pLg.score(X_train, y_train), pLg.score(X_test, y_test))

NameError: name 'StandardScaler' is not defined